# Seletion effect for kinematics model of individual halo from the selection in the inclination angles

In this notebook, we want to test, for an axisymmetric galaxy with certain intrinsic axis ratio, what is the average predicted velocity dispersion under random inclination angle or under lensing selected inclination angle. The goal is to show how lensing selection in the inclination angle affect our intepretation of the kinematics model.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import corner
from copy import deepcopy

from deproject.Profiles.SIS_truncated_physical import SIS_truncated_physical
from deproject.Util.orientation import Isotropic_inclination
from deproject.Profiles.Hernquist import Hernquist
from deproject.Cosmo.default_cosmo import get_default_lens_cosmo
from deproject.MGE_analysis.mge_proj import MGE_Proj
from deproject.MGE_analysis.intr_mge import Intr_MGE
from deproject.MGE_analysis.mge_misc import sum_gaussian_components
from deproject.Util.ellipticity import Axis_ratio2ellipticity

from jampy.jam_axi_proj import jam_axi_proj
from jampy.mge_half_light_isophote import mge_half_light_isophote

from lenstronomy.Analysis.lens_profile import LensProfileAnalysis

from hierarc.Util.distribution_util import PDFSampling

In [ ]:
def get_truncsis_intr_mge(sigma_v, rc_kpc, r_sym, qintr, plot_mge=0, fignum=1):
    """get the amplitude and dispersion of the MGE describing the INTRINSIC mass density/stellar light profile along the symmetry axis 

    Args:
        sigma_v (_type_): sigma_sph of the truncated SIS profile
        rc_kpc (_type_): truncation radius in kpc
        r_sym (_type_): coordinate along the symmetry axis
        qintr (_type_): intrinsic axis ratio. If oblate, qintr < 1; if prolate, qintr > 1
        plot_mge (int, optional): _description_. Defaults to 0.
        fignum (int, optional): _description_. Defaults to 1.

    Returns:
        _type_: amplitude in M_sun/pc^3
        _type_: dispersion in pc
    """
    sis_profile = SIS_truncated_physical(sigma_v=sigma_v, rc = rc_kpc)
    intr_mge = Intr_MGE(profile=sis_profile, qintr=qintr, r_sym=r_sym)
    peak, sigma = intr_mge.MGE_param(kwargs_mge={'ngauss': 20, 'inner_slope': 3, 'outer_slope':1}, plot_mge=plot_mge, fignum=fignum)

    peak = peak / 1e9 # convert to [M_sun/pc^3]
    sigma = sigma * 1e3 # convert to [pc]

    return peak, sigma

def get_proj_mge(mge_proj, distance, inc):
    """get projected MGE

    Args:
        mge_proj (_type_): MGE_proj instance
        distance (_type_): angular diameter distance in Mpc, used to convert the dispersion to arcsec
        inc (_type_): inclination angle [rad]

    Returns:
        _type_: peak of the projected MGE [M_sun/pc^2]
        _type_: sigma of the projected MGE [arcsec]
        _type_: projected axis ratio 
    """
    surf = mge_proj.surf(inc=inc)
    qobs = mge_proj.qobs(inc=inc)

    qobs = np.full_like(surf, qobs)

    pc = distance * np.pi / 0.648 # convert to arcsec
    sigma_intr = mge_proj.sigma
    sigma = sigma_intr / pc

    return surf, sigma, qobs

def get_hernquist_intr_mge(m_star, Re_kpc, qintr, plot_mge=0, fignum=1):

    Rs_kpc = Re_kpc * 0.551
    hernquist_profile = Hernquist(Rs=Rs_kpc, sigma0=1e7)

    r_sym = np.geomspace(0.001, 10 * Rs_kpc, 300)

    intr_mge = Intr_MGE(profile=hernquist_profile, qintr=qintr, r_sym=r_sym)

    peak, sigma = intr_mge.MGE_param(kwargs_mge = {'ngauss': 20}, plot_mge=plot_mge, fignum=fignum)

    peak = peak / 1e9 # convert to [M_sun/pc^3]
    sigma = sigma * 1e3 # convert to [pc]

    mtot = intr_mge.MGE_mass_sph()
    peak = m_star / mtot * peak # rescale to desired input stellar mass

    return peak, sigma

def get_sigma_e(surf_lum, sigma_lum, qobs_lum, jam, xbin, ybin):
    """calculate velocity dispersion within the half-light radius from a jam model

    Args:
        surf_lum (_type_): peak of surface luminosity MGE 
        sigma_lum (_type_): sigma of surface luminocity MGE
        qobs_lum (_type_): array of the projected axis raio of the surface luminosity MGEs
        jam (_type_): jam model, a jampy.jam_axi_proj instance
        xbin (_type_): x coordinate to sample the velocity dispersion
        ybin (_type_): y coordinate to sample the velocity dispersion
        plot_velmap (int, optional): whether to plot the velocity dispersion map. Defaults to 0.
        plot_sample_points (int, optional): whether to plot the xy coordinates within the half-light radius. Defaults to 0.
        fignum (int, optional): _description_. Defaults to 1.

    Raises:
        ValueError: _description_

    Returns:
        _type_: _description_
    """
    ifu_dim = int(np.sqrt(len(xbin)))
    if np.all(qobs_lum <= 1):
        flux = jam.flux
        reff, reff_maj, eps_e, lum_tot = mge_half_light_isophote(surf_lum, sigma_lum, qobs_lum)
    elif np.all(qobs_lum > 1):
        flux = np.reshape(jam.flux, (ifu_dim, ifu_dim)).T  # for prolate rotate the flux map by 90 degrees to calculate the half-light radius
        flux = flux.flatten() 
        reff, reff_maj, eps_e, lum_tot = mge_half_light_isophote(surf_lum, sigma_lum, 1/qobs_lum)
    else:
        raise ValueError('Apparent axis ratio must be constant with radius!')

    w = xbin**2 + (ybin/(1 - eps_e))**2 < reff_maj**2

    model = jam.model

    sig_e = np.sqrt((flux[w]*model[w]**2).sum()/flux[w].sum())

    return sig_e



## Access random and selected inclination angle

In [ ]:
oblate = False

if oblate:
    oblate_name = 'oblate'
else:
    oblate_name = 'prolate'

In [ ]:
data_sample = np.load('../06-make_kin_mock_data/kin_mock_data/data_{}.npy' .format(oblate_name))
inc_deg_sample = data_sample[3, :] # access random inclination angle
theta_e_sample = data_sample[0, :]

theta_e_cut = 0.6 # select on the Einstein radius
ind = theta_e_sample >= theta_e_cut
inc_deg_select = inc_deg_sample[ind]
inc_select = np.radians(inc_deg_select)

# histogram of selected inclination angle
bin_num = 20
counts, bins = np.histogram(inc_select, bins = bin_num, density = True) 

# access the oblate catalog
cat = pd.read_pickle('../06-make_kin_mock_data/axisym_tng_catalog/{}_catalog.pkl' .format(oblate_name))

# set up a cosmology
lens_cosmo = get_default_lens_cosmo()
distance = lens_cosmo.Dd
sigma_crit = lens_cosmo.Sigma_crit / 1e12 # [M_sun/pc^2]

# number of projection for each individual galaxy
num_proj = 200

# loop over the catalog objects and inclinations
for i in range(len(cat)): # run only the first 20 sample

    gal_kwargs = cat.iloc[i].to_dict()
    halo_id = int(gal_kwargs.get('halo_id'))

    # sample selected inclination angle 
    pdfsampling = PDFSampling(bin_edges=bins, pdf_array=counts)
    inc_sel = pdfsampling.draw(num_proj)
    # create empty arrays to store data
    sigma_e_all, qobs_all, theta_e_all = np.zeros(shape = (3, num_proj))
    sigma_e_rm_all, qobs_rm_all, theta_e_rm_all = np.zeros(shape = (3, num_proj))

    sigma_rm = gal_kwargs.get('sigma_random_los')
    Re_kpc = gal_kwargs.get('Re')
    if oblate:
        qintr = gal_kwargs.get('xi_new')
    else:
        qintr = 1 / gal_kwargs.get('zeta_new')

    # compute rc_kpc for truncated sis profile
    theta_e_sis = lens_cosmo.sis_sigma_v2theta_E(sigma_rm)
    theta_e_sis_kpc = lens_cosmo.arcsec2Mpc_lens(theta_e_sis) * 1e3
    rc_kpc = theta_e_sis_kpc * 200
    # compute effective radius in arcsec
    Re_arcsec = lens_cosmo.Mpc2arcsec_lens(Re_kpc / 1e3)

    # sample inclination angle uniformly on a sphere
    inc_rm_all = Isotropic_inclination(num_proj)
    inc_rm_deg_all = np.degrees(inc_rm_all)

    # intrinsic MGE component of density and luminosity
    r_sym = np.geomspace(0.01, 5 * rc_kpc, 300)
    r_fine = np.geomspace(0.005, 10 * theta_e_sis, 100)
    peak_den, sigma_den = get_truncsis_intr_mge(sigma_rm, rc_kpc, r_sym, qintr)
    peak_lum, sigma_lum = get_hernquist_intr_mge(1e11, Re_kpc, qintr)
    # isotropic anisotropy parameter
    beta = np.zeros_like(peak_lum)

    # projection of the mge
    mge_proj_den = MGE_Proj(peak_den, sigma_den, qintr)
    mge_proj_lum = MGE_Proj(peak_lum, sigma_lum, qintr)

    # make a grid
    xx = np.linspace(-2.5 * Re_arcsec, 2.5 * Re_arcsec, 100)  # avoid (x,y)=(0,0)
    xbin, ybin = map(np.ravel, np.meshgrid(xx, xx))

    # loop over isotropic inclination angle
    for j in range(num_proj):

        inc_rm = inc_rm_all[j]
        inc_rm_deg = inc_rm_deg_all[j]

        # first solve for random inclination
        peak_surf_den, sigma_surf_den, qobs_den = get_proj_mge(mge_proj_den, distance, inc_rm)
        peak_surf_lum, sigma_surf_lum, qobs_lum = get_proj_mge(mge_proj_lum, distance, inc_rm)

        # compute observed axis ratio 
        Qobs = mge_proj_den.qobs(inc=inc_rm)
        qobs_rm_all[j] = Qobs

        # compute einstein radius from the radial profile
        surf_den_circ = sum_gaussian_components(r_fine/np.sqrt(Qobs), peak_surf_den, sigma_surf_den) # circularized surface density profile
        theta_e_jam = LensProfileAnalysis.effective_einstein_radius_from_radial_profile(r_fine, surf_den_circ/sigma_crit)
        theta_e_rm_all[j] = theta_e_jam
        print(f"theta_E_rm: {theta_e_jam:.3f} arcsec")

        # jam prediction of LoS velocity dispersion
        jam = jam_axi_proj(peak_surf_lum, sigma_surf_lum, qobs_lum, peak_surf_den, sigma_surf_den, qobs_den, inc_rm_deg, 0, distance, xbin, ybin, plot=0, beta=beta, align='sph')
        sigma_e = get_sigma_e(peak_surf_lum, sigma_surf_lum, qobs_lum, jam, xbin, ybin)
        print(f"sigma_e_rm: {sigma_e:.2f} km/s")
        sigma_e_rm_all[j] = sigma_e

    if oblate:
        eobs_rm_all = Axis_ratio2ellipticity(qobs_rm_all)
    else:
        eobs_rm_all = -1 * Axis_ratio2ellipticity(qobs_rm_all)

    data_rm = np.vstack([theta_e_rm_all, eobs_rm_all, sigma_e_rm_all, inc_rm_deg_all])

    np.save('./data_{}/{}_isotropic_{}.npy' .format(oblate_name, oblate_name, halo_id), data_rm)

    # loop over selected inclination angle
    for j in range(num_proj):

        inc = inc_sel[j]

        peak_surf_den, sigma_surf_den, qobs_den = get_proj_mge(mge_proj_den, distance, inc)
        peak_surf_lum, sigma_surf_lum, qobs_lum = get_proj_mge(mge_proj_lum, distance, inc)

        # compute observed axis ratio 
        Qobs = mge_proj_den.qobs(inc=inc)
        qobs_all[j] = Qobs

        # compute einstein radius from the radial profile
        surf_den_circ = sum_gaussian_components(r_fine/np.sqrt(Qobs), peak_surf_den, sigma_surf_den) # circularized surface density profile
        theta_e_jam = LensProfileAnalysis.effective_einstein_radius_from_radial_profile(r_fine, surf_den_circ/sigma_crit)
        theta_e_all[j] = theta_e_jam
        print(f"theta_E: {theta_e_jam:.3f} arcsec")

        # jam prediction of LoS velocity dispersion
        jam = jam_axi_proj(peak_surf_lum, sigma_surf_lum, qobs_lum, peak_surf_den, sigma_surf_den, qobs_den, np.degrees(inc), 0, distance, xbin, ybin, plot=0, beta=beta, align='sph')
        sigma_e = get_sigma_e(peak_surf_lum, sigma_surf_lum, qobs_lum, jam, xbin, ybin)
        print(f"sigma_e: {sigma_e:.2f} km/s")
        sigma_e_all[j] = sigma_e

    if oblate:
        eobs_all = Axis_ratio2ellipticity(qobs_all)
    else:
        eobs_all = -1 * Axis_ratio2ellipticity(qobs_all)

    data = np.vstack([theta_e_all, eobs_all, sigma_e_all, np.degrees(inc_sel)])

    np.save('./data_{}/{}_selected_{}.npy' .format(oblate_name, oblate_name, halo_id), data)